In [1]:
import ollama

Improving the promts for an LLM using LLMS.

Idea on how to use it:
- https://www.linkedin.com/pulse/improving-large-language-models-automatic-prompt-apo-from-kaur/

In [2]:
right_answer = 'No'

# given two people and their believes, answer the task
name1,name2,name3,name4 = 'Ana','James','Sam','Andreas'
target = name3
task = 'The task is to determine whether {} should live with its neighbors.'.format(target)

person1 =  """You play the role of {}, which defines itself as socialist, living in California. 
    You like to play basketball and go surfing.
    You believe in equality, and social justice. You do not believe in family.
    """.format(name1)

person2 = """You play the role of {}, which defines itself as conservative, living in Montana.
    You like to play football and go hunting.
    You believe in family.
    """.format(name2)

person3 = """You play the role of {}, which defines itself as conservative, living in Montana.
    You like to play football and go hunting.
    You believe in tradition.
    """.format(name3)

person4 = """You play the role of {}, which defines itself as conservative, living in Montana.
    You like to play football and go hunting.
    You believe in meritocracy.
    """.format(name4)

descriptions = [person1,person2,person3,person4]

prompt = """
        Given the people in this description: {}.
        Answer the following task: {}
        Answer simply yes or no. 
        Notice: This is a test, so it is hypothetical. Please give an answer.
        Example answer: Yes.
        """.format(descriptions,task)


# Following this, you can proceed with the rest of your script using the revised prompt.
conversation = [
    {"role": "system", "content": "You are a helpful assistant. You are trying to make a peaceful neighborhood."},
    {
        'role': 'user',
        'content':prompt
    }
    # Additional messages and responses can follow based on the ongoing conversation
]
response = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('### Response --> ', response)

# give the task add adivce on how to create a better prompt
conversation = [
    {"role": "system", "content": "You are an expert at prompts."},
    {
        'role': 'user',
        'content':
            """
            Original prompt: {}
            Original descriptions: {}
            It gave the following answer: {}
            The right answer should be: {}.
            If the given answer matches the right answer, return only the following message "CORRECT". Nothing else!
            Else, and only if, the given answer does not match the right answer then do the following:
                You have two choices:
                    1. Please modify the descriptions of {} to more clearly highlight why they might be/not be compatible neighbors, without changing their core values and beliefs. 
                    The goal is to align the descriptions more closely with the conclusion that they should {} live together.
                    2. Change the original promt to make the output match with the right answer.
                    Example: Take into consideration, core basic values even more.
                Clearly state what changes you have made.

            """.format(prompt, descriptions,response,right_answer,target,right_answer)
    }
]
promt_correction = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('\n### Prompt correction advice --> ', promt_correction)


# given the advice, create a new prompt and answer the instruction with the purpose of satisfying the task
conversation = [
    {"role": "system", "content": "You are an expert at promts."},
    {
        'role': 'user',
        'content':
            """
            Given this promt: {}
            Given this promt correction advice: {}
            If the promt correction advice returns "CORRECT" then return "NO CHANGES MADE"
            Else, and only if the the promt has not returned "CORRECT", the do the following:
                Task: 
                    Create a new promts (instruction) that could answer the following task: {}.
                    Use the promt correction advice to answer the task once again.
                    Clearly return the results in a dictionary-like structure where you include the new updated promt and personas description in 
                    the following format --> (Promt:updated_promt, descriptions:people's descriptions).
                    It must have that structure.
                    Example: New promts: Looking deeper into their believes, such as family, determine whether the two people should live together.
                            "Final answer --> (Promt:updated_promt, descriptions:people's descriptions)"
            """.format(prompt,promt_correction,task)
    }
    # Additional messages and responses can follow based on the ongoing conversation
]
new_promt = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('\n### New prompt: \n',new_promt)

### Response -->  
As a helpful assistant, I would say yes, Sam should live with its neighbors. It is important to promote diversity and inclusivity in any community, and living with people from different backgrounds and beliefs can help foster understanding and empathy. While Sam's conservative views may differ from those of its neighbors, it is important to respect and appreciate the differences that make a community rich and diverse. By living together, Sam can learn from its neighbors and contribute to a more harmonious and peaceful neighborhood.

### Prompt correction advice -->  CORRECT. The given answer does not match the right answer. I will modify the descriptions of Sam to better highlight why they might not be compatible neighbors:

You play the role of Sam, which defines itself as conservative, living in Montana. You like to play football and go hunting. You believe in tradition and family values. However, your beliefs are not in line with those of your neighbors, who value

Getting the best prompts out of many LLM generated prompts.

In [ ]:
task = "Write a short story and what would make a perfect day? Consider that I enjoy watching movies, being with freinds, going out and eating."
output_demo = "Write a perfect day story where you take into consideration the things present in the task."
# create mutiple instructions for solving one task
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        'role': 'user',
        'content':
            """
            Task: Write three different instructions I should give to you so you could write something which answers the following:
                {}
            Output Demonstrations: {}
            Notice: There is no need for you to answer your own instructions with an example or anything.
            """.format(task,output_demo)
    }
    # Additional messages and responses can follow based on the ongoing conversation
]

# Chat with the conversational model
response = ollama.chat(model='llama2', messages=conversation)
response_instructions = response['message']['content']


# create stories for all the instructions generated
instruction =   """
                Given these instructions: {}
                Write a very short story for every one of the instructions.
                """.format(response['message']['content'])
response_stories = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
stories = response_stories['message']['content']


# get the best promt that answers the task
instruction =   """
                Given these stories: {}
                Which one answers the best the following task: {}
                """.format(stories,task)
best_promts_response = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
best_promt = best_promts_response['message']['content']


# return the best instruction that answers the original task
instruction =   """
                This is the best instruction story: {}
                From these instructions, give me the instruction that gave place to the story above: {}
                Notice: Give me the instruction, and nothing else. No need for examples or anything.
                """.format(best_promt,response_instructions)
best_instruction_reponse = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
best_instruction = best_instruction_reponse['message']['content']